In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math
import re
from datetime import datetime



In [2]:
r = requests.get('https://www.airlinequality.com/airline-reviews/british-airways')
r.status_code

200

In [3]:
html = r.content
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7 lt-ie10" lang="en-GB"> <![endif]-->
<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8 lt-ie10" lang="en-GB"> <![endif]-->
<!--[if IE 8]>    <html class="no-js lt-ie9 lt-ie10" lang="en-GB"> <![endif]-->
<!--[if IE 9]>    <html class="no-js lt-ie10" lang="en-GB"> <![endif]-->
<!--[if gt IE 8]><!-->
<html lang="en-GB">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <title>
   British Airways Customer Reviews - SKYTRAX
  </title>
  <link as="image" data-rocket-preload="" fetchpriority="high" href="https://www.airlinequality.com/wp-content/themes/airlinequality2014new/library/images/nav/grad-header.jpg" rel="preload"/>
  <!-- Google Chrome Frame for IE -->
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <!-- mobile meta -->
  <meta content="True" name="HandheldFriendly"/>
  <meta content="320" name="MobileOptimized"/>
  <meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0

class:
text_content 

table review-ratings

In [4]:
total_results = int(soup.find('article', attrs={'class': 'comp comp_reviews-pagination querylist-pagination position-'}) \
                    .get_text(strip=True).split()[-2])
total_results

3924

In [5]:
no_pages = math.ceil(total_results / 10)  
no_pages

393

In [6]:
starts = range(1, no_pages)  
starts

range(1, 393)

In [7]:
html_table = soup.find_all('table', attrs={'class': 'review-ratings'})[0]
html_table

<table class="review-ratings">
<tr>
<td class="review-rating-header food-beverages">Food &amp; Beverages</td>
<td class="review-rating-stars stars">
<span class="star fill">1</span><span class="star fill">2</span><span class="star fill">3</span><span class="star">4</span><span class="star">5</span> </td>
</tr>
<tr>
<td class="review-rating-header inflight-entertainment">Inflight Entertainment</td>
<td class="review-rating-stars stars">
<span class="star fill">1</span><span class="star fill">2</span><span class="star fill">3</span><span class="star">4</span><span class="star">5</span> </td>
</tr>
<tr>
<td class="review-rating-header seat-comfort">Seat Comfort</td>
<td class="review-rating-stars stars">
<span class="star fill">1</span><span class="star fill">2</span><span class="star fill">3</span><span class="star">4</span><span class="star">5</span> </td>
</tr>
<tr>
<td class="review-rating-header staff-service">Staff Service</td>
<td class="review-rating-stars stars">
<span class="sta

In [8]:
reviews = soup.find_all('article', attrs={'itemprop': "review"})
reviews[0]

<article class="comp comp_media-review-rated list-item media position-content review-921004" itemprop="review" itemscope="" itemtype="http://schema.org/Review">
<meta content="2025-03-19" itemprop="datePublished"/>
<div class="rating-10" itemprop="reviewRating" itemscope="" itemtype="http://schema.org/Rating">
<span itemprop="ratingValue">7</span>/<span itemprop="bestRating">10</span>
</div>
<div class="body" id="anchor921004">
<h2 class="text_header">"disagreeable flight attendant" </h2>
<h3 class="text_sub_header userStatusWrapper">
<span itemprop="author" itemscope="" itemtype="http://schema.org/Person">
<a class="userStatus hiflyer" href="/reviewer/reviewer-signup"><span class="userStatusReviewCount">10 reviews</span><svg class="userStatusIcon" version="1.1" viewbox="0 0 307.3 193.5" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">
<path d="M157.9,95c-13.1,5.5-26.3,11.1-38.5,16c-21,8.4-42.4,16-65.1,18.7c-16,1.5-26.5-1-31.6-9.7C15.6,104.3,8.2,88.7,1.1,7

In [9]:
def get_reviews(reviews):    
    # Initialize lists for storing extracted data
    dates_published = []
    reviews_text = []
    aircrafts = []
    type_of_travellers = []
    seat_types = []
    routes = []
    dates_flown = []
    seat_comfort_stars = []
    cabin_staff_service_stars = []
    food_beverages_stars = []
    ground_services_stars = []
    value_for_money_stars = []
    recommended = []
    
    for review in reviews:

        date_published = review.find('time', attrs={'itemprop': "datePublished"})
        dates_published.append(date_published.get_text(strip=True) if date_published else None)
        
        review_text = review.find('div', class_="text_content")
        reviews_text.append(review_text.get_text(strip=True) if review_text else 'Not informed')

        aircraft = review.find('td', class_="review-rating-header aircraft")
        aircrafts.append(aircraft.find_next_sibling('td').get_text(strip=True) if aircraft else 'Not informed')

        traveller = review.find('td', class_="review-rating-header type_of_traveller")
        type_of_travellers.append(traveller.find_next_sibling('td').get_text(strip=True) if traveller else 'Not informed')

        seat_type = review.find('td', class_="review-rating-header cabin_flown")
        seat_types.append(seat_type.find_next_sibling('td').get_text(strip=True) if seat_type else 'Not informed')

        route = review.find('td', class_="review-rating-header route")
        routes.append(route.find_next_sibling('td').get_text(strip=True) if route else 'Not informed')

        date_flown = review.find('td', class_="review-rating-header date_flown")
        dates_flown.append(date_flown.find_next_sibling('td').get_text(strip=True) if date_flown else None)

        def extract_star_rating(category):
            stars = review.find('td', class_=f"review-rating-header {category}")
            return len(stars.find_next_sibling('td').find_all('span', class_="star fill")) if stars else None

        seat_comfort_stars.append(extract_star_rating("seat_comfort"))
        cabin_staff_service_stars.append(extract_star_rating("cabin_staff_service"))
        food_beverages_stars.append(extract_star_rating("food_and_beverages"))
        ground_services_stars.append(extract_star_rating("ground_service"))
        value_for_money_stars.append(extract_star_rating("value_for_money"))

        recommended_tag = review.find('td', class_="review-rating-header recommended")
        recommended.append(recommended_tag.find_next_sibling('td').get_text(strip=True) if recommended_tag else 'Not informed')

    dct = {
        'date_published': dates_published,
        'review_text': reviews_text,
        'aircraft': aircrafts,
        'type_of_traveller': type_of_travellers,
        'seat_type': seat_types,
        'route': routes,
        'date_flown': dates_flown,
        'seat_comfort_ranking': seat_comfort_stars,
        'cabin_staff_service_ranking': cabin_staff_service_stars,
        'food_beverages_ranking': food_beverages_stars,
        'ground_services_ranking': ground_services_stars,
        'value_for_money': value_for_money_stars,
        'recommended': recommended
    }
    
    return dct


In [10]:
df = pd.DataFrame()

for start in starts:
    r = requests.get(f'https://www.airlinequality.com/airline-reviews/british-airways/page/{start}/')
    soup = BeautifulSoup(r.content, 'html.parser')
    reviews = soup.find_all('article', attrs={'itemprop': "review"})
    info_dct = get_reviews(reviews)
    new_df = pd.DataFrame.from_dict(info_dct)
    df = pd.concat([df, new_df], ignore_index=True)

df.head()


,date_published,review_text,aircraft,type_of_traveller,seat_type,route,date_flown,seat_comfort_ranking,cabin_staff_service_ranking,food_beverages_ranking,ground_services_ranking,value_for_money,recommended
0,19th March 2025,✅Trip Verified| Flight mainly let down by a ...,Boeing 787,Solo Leisure,Business Class,Cape Town to London,March 2025,5.0,2.0,5.0,4.0,4,yes
1,16th March 2025,✅Trip Verified| Another awful experience by ...,Not informed,Business,Business Class,London to Geneva,December 2024,1.0,1.0,NaN,1.0,1,no
2,16th March 2025,"✅Trip Verified| The service was rude, full o...",Not informed,Business,Premium Economy,London to New York,January 2025,1.0,1.0,1.0,5.0,1,no
3,16th March 2025,✅Trip Verified| This flight was a joke. Ther...,Not informed,Business,Business Class,Chambery to London,January 2025,1.0,1.0,2.0,1.0,1,no
4,7th March 2025,✅Trip Verified| This time British Airways ma...,Boeing 777,Business,Premium Economy,Gatwick to Antigua,March 2025,5.0,5.0,4.0,5.0,5,yes


df = pd.DataFrame.from_dict(dct)
df

In [11]:
df.to_csv('skytrax-data.csv', index=False)